<a href="https://colab.research.google.com/github/efeoktistova/1C-to-Finantional-reports/blob/main/Data_for_CF_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Аннотация:
1. Чтение эксель, удаление пустых столбцов
2. Преобразование даты в дататайм, выделение месяца
3. Разделение столбцов, отбор информативнх, объединение датафремов (concat)
4. Загрузка словаря со статьями по контрагентам, добавление статьи упр.учета к датафрейму (map)
5. Отбор строк с выручкой, присоение статьи по условию в новом дф, замена строк в исх.дф
6. Аналогично со статьями по Финансовой деятельности.
8. Аналогично по статьям налоги и соц.взносов
9. Аналогично по строкам "Расходы на услуги банков"
10. Отбор столбцов и выгрузка в эксель



In [47]:
import pandas as pd
import numpy as np

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# Карточка счета 51 по форме 1С
df = pd.read_excel('', header = [5], skiprows=[6,7], dtype = {'Дебет': 'str', 'Кредит' : 'str'})
df.drop(df.tail(1).index,inplace=True) #удалить последнюю строку с итогами
df.drop(['Unnamed: 6', 'Unnamed: 9', 'Текущее сальдо', 'Unnamed: 11'], axis = 1, inplace = True )
df.head(0) #шапка

,Период,Документ,Аналитика Дт,Аналитика Кт,Дебет,Unnamed: 5,Кредит,Unnamed: 8


In [50]:
df = df.rename(columns = {'Unnamed: 5':'Сумма Дт','Unnamed: 8':'Сумма Кт'})
df.head(0)

,Период,Документ,Аналитика Дт,Аналитика Кт,Дебет,Сумма Дт,Кредит,Сумма Кт


In [51]:
# преобразование даты
df['Период'].dtypes

dtype('O')

In [52]:
# преобразование стр в дататайм, когда первым указано число
df['Период'] = pd.to_datetime(df['Период'], dayfirst = True)

In [53]:
df['Период'].dtypes

dtype('<M8[ns]')

In [54]:
# выделение месяца
df['Месяц'] = pd.to_datetime(df['Период']).dt.month
df.head(0)

,Период,Документ,Аналитика Дт,Аналитика Кт,Дебет,Сумма Дт,Кредит,Сумма Кт,Месяц


In [55]:
# проверка
df['Месяц'].unique()

array([10, 11, 12])

In [56]:
#разделение и отбор столбцов
new_df1 = df['Аналитика Дт'].str.split('\n',expand=True).reset_index()
new_df1.columns

Index(['index', 0, 1, 2], dtype='object')

In [57]:
new_df1.drop(['index',2], axis= 1 , inplace= True)
new_df1.columns = ['Аналитика Дт 1', 'Аналитика Дт 2']
new_df1.head(0)

,Аналитика Дт 1,Аналитика Дт 2


In [58]:
new_df2 = df['Аналитика Кт'].str.split('\n',expand=True).reset_index()
new_df2.columns

Index(['index', 0, 1, 2], dtype='object')

In [59]:
new_df2.drop(['index',2], axis= 1 , inplace= True)
new_df2.columns = ['Аналитика Кт 1', 'Аналитика Кт 2']
new_df2.head(0)

,Аналитика Кт 1,Аналитика Кт 2


In [60]:
new_df3 = df['Документ'].str.split('\n',expand=True).reset_index()
new_df3.columns

Index(['index', 0, 1], dtype='object')

In [61]:
new_df3.drop(['index',0], axis= 1 , inplace= True)
new_df3.columns = ['Док']
new_df3.head(0)

,Док


In [62]:
#new_df2['Аналитика Кт 3'].unique()

In [63]:
df = pd.concat([df,new_df3,new_df1,new_df2],axis=1)
df.drop(['Документ','Аналитика Дт', 'Аналитика Кт'],axis= 1 , inplace= True )
df.head(0)

,Период,Дебет,Сумма Дт,Кредит,Сумма Кт,Месяц,Док,Аналитика Дт 1,Аналитика Дт 2,Аналитика Кт 1,Аналитика Кт 2


In [64]:
#df['Аналитика Дт 1'].unique()

In [65]:
# загрузка инф о контрагентах и статьях ##таблица из 2х столбцов: контрагент - утвержденное наименование статьи CF, данные являются коммерческой тайной
ex = pd.read_excel('', header = [1])
ex.head(0)

,Сотрудник/контрагент,Статья,проект,детализация capex,детализация capex.1


In [66]:
# отбор столбцов
ex_cols = ['Сотрудник/контрагент', 'Статья']

In [67]:
ex = ex[ex_cols]
ex.head(0)

,Сотрудник/контрагент,Статья


In [68]:
# создание словаря из дф (выделение из дф)
ex_dict = dict(ex.values)
#ex_dict

In [69]:
#добавление нового столбца (статья УУ) из словаря
df['статья ДДС'] = df['Аналитика Дт 1'].map(ex_dict)
#df.sample(5)


In [70]:
#отбор строк с выручкой Дт51Кт62
temp1 = df.loc[((df['Дебет'] == '51') & ((df['Кредит'] == '62.01')|(df['Кредит'] == '62.02')))]
#temp1.tail(5)

In [71]:
# присвоение статьи ддс для выручки по условию, создаем список из вариантов
lst = ['20%', 'НДС:', 'медиа-комплект', 'в том числе','в т.ч.']
#case=False отключает чувствительность к регистру
temp1['статья ДДС']=np.where(temp1['Док'].str.contains('|'.join(lst), na=False, case=False),'Поступления от реализации сертификатов технической поддержки', 'Поступления от реализация ПО и прочей продукции')
#temp1.sample(20)

<ipython-input-71-4d2626a5d205>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1['статья ДДС']=np.where(temp1['Док'].str.contains('|'.join(lst), na=False, case=False),'Поступления от реализации сертификатов технической поддержки', 'Поступления от реализация ПО и прочей продукции')


In [72]:
#добавление статей по выручке в общий дф
df.loc[((df['Дебет'] == '51') & ((df['Кредит'] == '62.01')|(df['Кредит'] == '62.02'))), 'статья ДДС'] = temp1['статья ДДС']
df['статья ДДС'].unique()

array([nan, 'Поступления от реализация ПО и прочей продукции',
       'Выплаты по маркетингу и рекламе', 'Оплата аренды',
       'Выплаты по заработной плате и ФОТ',
       'Создание нематериальных активов',
       'Поступления от реализации сертификатов технической поддержки',
       'Оплата расходов на содержание офиса и производственных помещений',
       'Другие выплаты', 'Оплата расходов на IT',
       'Производственные выплаты (лиценз-ние, документация, медиакомплекты)',
       'Приобретение материалов',
       'Оплата консультационных, информационных и аудиторских услуг'],
      dtype=object)

In [73]:
df.loc[((df['Дебет'] == '51') & ((df['Кредит'] == '62.01')|(df['Кредит'] == '62.02'))), 'статья ДДС']

3         Поступления от реализация ПО и прочей продукции
15        Поступления от реализация ПО и прочей продукции
16      Поступления от реализации сертификатов техниче...
17        Поступления от реализация ПО и прочей продукции
18        Поступления от реализация ПО и прочей продукции
                              ...                        
1133    Поступления от реализации сертификатов техниче...
1134    Поступления от реализации сертификатов техниче...
1135    Поступления от реализации сертификатов техниче...
1213      Поступления от реализация ПО и прочей продукции
1280      Поступления от реализация ПО и прочей продукции
Name: статья ДДС, Length: 123, dtype: object

In [74]:
# прочие поступления, фильтр строк и добавление статьи
temp2 = df.loc[(df['Дебет'] == '51')&(df['Кредит'] != '62.01')&(df['Кредит'] != '62.02')]
temp2['Кредит'].unique()

array(['57.01', '91.01', '60.02', '76.09'], dtype=object)

In [75]:
temp2['статья ДДС']=np.where ((temp2['Кредит'] == '55.03')| (temp2['Кредит'] == '58.03'), 'Поступления от реализации/погашения краткосрочных ФВ',
                              np.where ((temp2['Кредит'] == '66.03')| (temp2['Кредит'] == '67.03'),'Поступление займов и кредитов от акционеров',
                                        np.where ((temp2['Кредит'] == '76.03')| (temp2['Кредит'] == '76.09'),'Проценты от размещения средств на депозитных счетах', '')))

<ipython-input-75-4949ad069cd2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['статья ДДС']=np.where ((temp2['Кредит'] == '55.03')| (temp2['Кредит'] == '58.03'), 'Поступления от реализации/погашения краткосрочных ФВ',


In [76]:
#temp2.sample(3)

In [77]:
df.loc[(df['Дебет'] == '51') & (df['Кредит'] != '62.01')& (df['Кредит'] != '62.02'), 'статья ДДС'] = temp2['статья ДДС']


In [78]:
df['статья ДДС'].unique()

array([nan, 'Поступления от реализация ПО и прочей продукции',
       'Выплаты по маркетингу и рекламе', 'Оплата аренды',
       'Выплаты по заработной плате и ФОТ',
       'Создание нематериальных активов',
       'Поступления от реализации сертификатов технической поддержки', '',
       'Оплата расходов на содержание офиса и производственных помещений',
       'Другие выплаты', 'Оплата расходов на IT',
       'Производственные выплаты (лиценз-ние, документация, медиакомплекты)',
       'Приобретение материалов',
       'Оплата консультационных, информационных и аудиторских услуг',
       'Проценты от размещения средств на депозитных счетах'],
      dtype=object)

In [79]:
# ндфл и соц взносы
temp3 = df.loc[(df['Дебет'].str.contains('69'))|(df['Дебет'].str.contains('68'))]
temp3['статья ДДС'] = 'Прочие налоги (НДФЛ, страховые взносы)'
#temp3.sample(3)

<ipython-input-79-629da8ac5b2f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp3['статья ДДС'] = 'Прочие налоги (НДФЛ, страховые взносы)'


In [80]:
# расходы на услуги банков
temp4 = df.loc[(df['Аналитика Дт 1'] == 'Расходы на услуги банков')]
temp4['статья ДДС'] = 'Другие выплаты'
#temp4.sample(3)

<ipython-input-80-7db20448a448>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp4['статья ДДС'] = 'Другие выплаты'


In [81]:
df.loc[(df['Дебет'].str.contains('69'))|(df['Дебет'].str.contains('68')), 'статья ДДС'] = temp3['статья ДДС']
df.loc[(df['Аналитика Дт 1'] == 'Расходы на услуги банков'), 'статья ДДС'] = temp4['статья ДДС']
#df.sample(10)


In [82]:
df['статья ДДС'].unique()

array(['Другие выплаты',
       'Поступления от реализация ПО и прочей продукции', nan,
       'Выплаты по маркетингу и рекламе', 'Оплата аренды',
       'Выплаты по заработной плате и ФОТ',
       'Создание нематериальных активов',
       'Поступления от реализации сертификатов технической поддержки', '',
       'Оплата расходов на содержание офиса и производственных помещений',
       'Оплата расходов на IT', 'Прочие налоги (НДФЛ, страховые взносы)',
       'Производственные выплаты (лиценз-ние, документация, медиакомплекты)',
       'Приобретение материалов',
       'Оплата консультационных, информационных и аудиторских услуг',
       'Проценты от размещения средств на депозитных счетах'],
      dtype=object)

In [83]:
# замена пропущенных значение на 0 и образование единого столбца с суммой
df[['Сумма Дт','Сумма Кт']]=df[['Сумма Дт','Сумма Кт']].fillna(0)
df['Сумма'] = df['Сумма Дт'] + df['Сумма Кт']
df.head(0)

,Период,Дебет,Сумма Дт,Кредит,Сумма Кт,Месяц,Док,Аналитика Дт 1,Аналитика Дт 2,Аналитика Кт 1,Аналитика Кт 2,статья ДДС,Сумма


In [84]:
# порядок столбцов
our_cols = ['Период','Месяц','Дебет','Кредит','Сумма','статья ДДС','Док','Аналитика Кт 1','Аналитика Кт 2','Аналитика Дт 1','Аналитика Дт 2']
df = df[our_cols]
df.head(0)

,Период,Месяц,Дебет,Кредит,Сумма,статья ДДС,Док,Аналитика Кт 1,Аналитика Кт 2,Аналитика Дт 1,Аналитика Дт 2


In [85]:
df.to_excel('/content/drive/MyDrive/Карточки _обработка Python/cf_myconsalid.xlsx', index = False)

In [86]:
df['статья ДДС'].unique()

array(['Другие выплаты',
       'Поступления от реализация ПО и прочей продукции', nan,
       'Выплаты по маркетингу и рекламе', 'Оплата аренды',
       'Выплаты по заработной плате и ФОТ',
       'Создание нематериальных активов',
       'Поступления от реализации сертификатов технической поддержки', '',
       'Оплата расходов на содержание офиса и производственных помещений',
       'Оплата расходов на IT', 'Прочие налоги (НДФЛ, страховые взносы)',
       'Производственные выплаты (лиценз-ние, документация, медиакомплекты)',
       'Приобретение материалов',
       'Оплата консультационных, информационных и аудиторских услуг',
       'Проценты от размещения средств на депозитных счетах'],
      dtype=object)